
# Cytochrome bc1 (chain C) — QoI ligands Docking & Visualization Workflow (rigid and flexible docking)

**Environment requirements (already in conda env):**
- `python=3.11`, `rdkit`, `openbabel` (CLI `obabel`), `vina` (AutoDock Vina), `biopython`, `mdanalysis`, `pdb2pqr` (pip), `meeko` (pip)
- Viz/analysis: `py3Dmol`, `nglview` (optional), `prolif`, `matplotlib`, `pandas`, `numpy`

**A comprehensive web source that can be useful: https://www.click2drug.org/**

> Tip: Run cells top-to-bottom. Adjust file paths in the first setup cell if needed.


# Cahya Prihatna

This notebook:
- Runs batch rigid docking
- Runs batch flexible
- Analyze contacts and weak forces in ligand-protein interaction
- Writes Coot-friendly complexes with ligand chain `Z`

# Rigid docking section

**Cell 00 — HPC Setup (CPU detection)**

In [1]:
import os, re, socket

def slurm_alloc_cpus(default=8, cap=16):
    v = os.environ.get("SLURM_CPUS_PER_TASK")
    if v and v.isdigit():
        return min(int(v), cap)

    v = os.environ.get("SLURM_CPUS_ON_NODE")
    if v and v.isdigit():
        return min(int(v), cap)

    v = os.environ.get("SLURM_JOB_CPUS_PER_NODE")
    if v:
        m = re.match(r"(\d+)", str(v))
        if m:
            return min(int(m.group(1)), cap)

    return min(os.cpu_count() or default, cap)

HOST = socket.gethostname()
JOB  = os.environ.get("SLURM_JOB_ID", None)
CPU_TO_USE = slurm_alloc_cpus(default=8, cap=16)

print("HOST:", HOST)
print("SLURM_JOB_ID:", JOB)
print("SLURM_CPUS_PER_TASK:", os.environ.get("SLURM_CPUS_PER_TASK"))
print("SLURM_CPUS_ON_NODE:", os.environ.get("SLURM_CPUS_ON_NODE"))
print("SLURM_JOB_CPUS_PER_NODE:", os.environ.get("SLURM_JOB_CPUS_PER_NODE"))
print("CPU_TO_USE:", CPU_TO_USE)


HOST: epyc008
SLURM_JOB_ID: 3541227
SLURM_CPUS_PER_TASK: None
SLURM_CPUS_ON_NODE: 32
SLURM_JOB_CPUS_PER_NODE: 32
CPU_TO_USE: 16


**Cell 01 — OPTIONAL: user-space micromamba + OpenBabel (only if you need obabel elsewhere)**

In [ ]:
%%bash
set -euo pipefail

MM="$HOME/.local/bin/micromamba"
MAMBA_ROOT_PREFIX="$HOME/.micromamba"
ENVNAME="dockobabel"

mkdir -p "$HOME/.local/bin"

if [ ! -x "$MM" ]; then
  cd /tmp
  if command -v curl >/dev/null 2>&1; then
    curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
  else
    wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
  fi
  mv -f bin/micromamba "$MM"
  chmod +x "$MM"
fi

if ! "$MM" -r "$MAMBA_ROOT_PREFIX" env list | awk '{print $1}' | grep -qx "$ENVNAME"; then
  "$MM" -r "$MAMBA_ROOT_PREFIX" create -y -n "$ENVNAME" -c conda-forge openbabel
fi

"$MM" -r "$MAMBA_ROOT_PREFIX" run -n "$ENVNAME" obabel -V
echo "OpenBabel OK"


**Cell 02 — Tool discovery (Vina + MGLTools modules; set paths)**

In [2]:
import os, subprocess
from pathlib import Path

CPU_TO_USE = int(globals().get("CPU_TO_USE", 16))

def which_from_module(module_name: str, exe: str):
    cmd = f'bash -lc "module purge >/dev/null 2>&1; module load {module_name} >/dev/null 2>&1; which {exe} || true"'
    r = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    p = r.stdout.strip()
    return p if p else None

# ---- Vina ----
vina_path = which_from_module("AutoDock-Vina", "vina")
if not vina_path:
    for mod in ["autodock-vina", "vina", "AutoDock_Vina", "AutoDock-Vina/1.1.2-linux_x86"]:
        vina_path = which_from_module(mod, "vina")
        if vina_path:
            break

# fallback (your known path)
if not vina_path:
    guess = Path("/mnt/shared/moduleapps/eb/AutoDock-Vina/1.1.2-linux_x86/bin/vina")
    if guess.exists():
        vina_path = str(guess)

# ---- MGLTools ----
pythonsh_path = which_from_module("mgltools", "pythonsh")
if not pythonsh_path:
    for mod in ["MGLTools", "mgltools/1.5.7", "mgltools-1.5.7"]:
        pythonsh_path = which_from_module(mod, "pythonsh")
        if pythonsh_path:
            break

print("vina     :", vina_path or "NOT FOUND")
print("pythonsh :", pythonsh_path or "NOT FOUND")
assert vina_path, "Vina not found (module or path)."
assert pythonsh_path, "MGLTools pythonsh not found (module)."

VINA_EXE = str(Path(vina_path).resolve())
PYTHONSH = str(Path(pythonsh_path).resolve())

# Locate Utilities24 scripts
mgl_root = Path(PYTHONSH).resolve().parents[1]
UTIL24 = mgl_root / "MGLToolsPckgs" / "AutoDockTools" / "Utilities24"
PREP_REC  = UTIL24 / "prepare_receptor4.py"
PREP_FLEX = UTIL24 / "prepare_flexreceptor4.py"

assert PREP_REC.exists(), f"Missing: {PREP_REC}"
assert PREP_FLEX.exists(), f"Missing: {PREP_FLEX}"

print("VINA_EXE :", VINA_EXE)
print("PYTHONSH :", PYTHONSH)
print("PREP_REC :", PREP_REC)
print("PREP_FLEX:", PREP_FLEX)

globals().update(dict(
    CPU_TO_USE=CPU_TO_USE,
    VINA_EXE=VINA_EXE,
    PYTHONSH=PYTHONSH,
    PREP_REC=str(PREP_REC),
    PREP_FLEX=str(PREP_FLEX),
))


vina     : /mnt/shared/moduleapps/eb/AutoDock-Vina/1.1.2-linux_x86/bin/vina
pythonsh : /mnt/shared/moduleapps/eb/mgltools/1.5.7/bin/pythonsh
VINA_EXE : /mnt/shared/moduleapps/eb/AutoDock-Vina/1.1.2-linux_x86/bin/vina
PYTHONSH : /mnt/shared/moduleapps/eb/mgltools/1.5.7/bin/pythonsh
PREP_REC : /mnt/shared/moduleapps/eb/mgltools/1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py
PREP_FLEX: /mnt/shared/moduleapps/eb/mgltools/1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_flexreceptor4.py


**Cell 03 — Shared constants + helpers (rigid+flex; Coot-safe complex stitching)**
*Some ligands may be weird due to vina writing, but will try to fix it later in the cell*

In [3]:
import re, math, subprocess
from pathlib import Path
import numpy as np
import pandas as pd

AA3 = {
    "ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE",
    "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL",
    "HSD","HSE","HSP","MSE"
}
WAT = {"HOH","WAT","H2O"}
HALOGENS = {"F","CL","BR","I"}
METALS   = {"ZN","FE","MN","MG","CA","CU","CO","NI"}

# AutoDock atom type -> element (IMPORTANT for Coot)
ADTYPE_TO_ELEM = {
    "A": "C",   # aromatic carbon
    "C": "C",
    "N": "N",
    "O": "O",
    "S": "S",
    "P": "P",
    "F": "F",
    "CL": "CL",
    "BR": "BR",
    "I": "I",
    "NA": "N",  # nitrogen acceptor
    "OA": "O",  # oxygen acceptor
    "SA": "S",  # sulfur acceptor
    "HD": "H",
}

def _infer_element_from_atom(atom_name: str) -> str:
    a = (atom_name or "").strip().upper()
    a = re.sub(r"[^A-Z0-9]", "", a)
    if not a:
        return "C"
    if a.startswith("CL"): return "CL"
    if a.startswith("BR"): return "BR"
    if a.startswith("NA"): return "NA"
    if a.startswith("MG"): return "MG"
    if a.startswith("ZN"): return "ZN"
    if a.startswith("FE"): return "FE"
    if a.startswith("MN"): return "MN"
    if a.startswith("CA"): return "CA"
    if a.startswith("CU"): return "CU"
    if a.startswith("CO"): return "CO"
    if a.startswith("NI"): return "NI"
    if a.startswith("SE"): return "SE"
    if a.startswith("I"):  return "I"
    return a[0]

def _elem_from_pdbqt_line(atom_name: str, pdbqt_line: str) -> str:
    toks = (pdbqt_line or "").split()
    atype = toks[-1].upper() if toks else ""
    if atype in ADTYPE_TO_ELEM:
        return ADTYPE_TO_ELEM[atype]
    return _infer_element_from_atom(atom_name)

def safe_lig_resname_from_path(p: str) -> str:
    """
    3-letter ligand resname that will NEVER collide with amino acids (AA3) or water.
    This fixes the 'met...' ligands becoming 'MET' (methionine) which breaks Coot + your AA3 filters.
    """
    stem = Path(p).stem.upper()
    stem = re.sub(r"[^A-Z0-9]", "", stem)
    raw = (stem + "XXX")[:3]
    if raw in AA3 or raw in WAT:
        raw = ("L" + raw[:2])[:3]  # e.g. MET -> LME
    return raw

def read_xyz_from_pdb(pdb_path: str):
    xs, ys, zs = [], [], []
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if ln.startswith(("ATOM","HETATM")) and len(ln) >= 54:
                try:
                    xs.append(float(ln[30:38])); ys.append(float(ln[38:46])); zs.append(float(ln[46:54]))
                except Exception:
                    pass
    return np.array(xs), np.array(ys), np.array(zs)

def qc_box_intersection_pdb(pdb_path: str, center, size) -> int:
    rx, ry, rz = read_xyz_from_pdb(pdb_path)
    if rx.size == 0:
        return 0
    cx, cy, cz = map(float, center)
    sx, sy, sz = map(float, size)
    hx, hy, hz = sx/2, sy/2, sz/2
    return int(np.sum(
        (cx-hx <= rx) & (rx <= cx+hx) &
        (cy-hy <= ry) & (ry <= cy+hy) &
        (cz-hz <= rz) & (rz <= cz+hz)
    ))

# ---- Vina helpers ----
def write_vina_cfg_rigid(cfg_path: Path, receptor_pdbqt: Path, ligand_pdbqt: Path,
                         center, size, out_pdbqt: Path,
                         exhaustiveness=16, num_modes=9, energy_range=4, seed=42, cpu=8, log_path=None):
    cx, cy, cz = map(float, center)
    sx, sy, sz = map(float, size)
    lines = [
        f"receptor = {Path(receptor_pdbqt)}",
        f"ligand   = {Path(ligand_pdbqt)}",
        "",
        f"center_x = {cx:.3f}",
        f"center_y = {cy:.3f}",
        f"center_z = {cz:.3f}",
        "",
        f"size_x = {sx:.2f}",
        f"size_y = {sy:.2f}",
        f"size_z = {sz:.2f}",
        "",
        f"exhaustiveness = {int(exhaustiveness)}",
        f"num_modes      = {int(num_modes)}",
        f"energy_range   = {int(energy_range)}",
        f"seed           = {int(seed)}",
        f"cpu            = {int(cpu)}",
        f"out            = {Path(out_pdbqt)}",
    ]
    if log_path:
        lines.append(f"log = {Path(log_path)}")
    Path(cfg_path).write_text("\n".join(lines) + "\n", encoding="utf-8")

def write_vina_cfg_flex(cfg_path: Path, receptor_pdbqt: Path, flex_pdbqt: Path, ligand_pdbqt: Path,
                        center, size, out_pdbqt: Path, log_path: Path,
                        exhaustiveness=16, num_modes=9, energy_range=4, seed=42, cpu=8):
    cx, cy, cz = map(float, center)
    sx, sy, sz = map(float, size)
    lines = [
        f"receptor = {Path(receptor_pdbqt)}",
        f"flex     = {Path(flex_pdbqt)}",
        f"ligand   = {Path(ligand_pdbqt)}",
        "",
        f"center_x = {cx:.3f}",
        f"center_y = {cy:.3f}",
        f"center_z = {cz:.3f}",
        "",
        f"size_x = {sx:.2f}",
        f"size_y = {sy:.2f}",
        f"size_z = {sz:.2f}",
        "",
        f"exhaustiveness = {int(exhaustiveness)}",
        f"num_modes      = {int(num_modes)}",
        f"energy_range   = {int(energy_range)}",
        f"seed           = {int(seed)}",
        f"cpu            = {int(cpu)}",
        f"out            = {Path(out_pdbqt)}",
        f"log            = {Path(log_path)}",
    ]
    Path(cfg_path).write_text("\n".join(lines) + "\n", encoding="utf-8")

def run_vina(cfg_path: Path):
    cmd = [VINA_EXE, "--config", str(cfg_path)]
    r = subprocess.run(cmd, text=True, capture_output=True)
    if r.returncode != 0:
        print("STDOUT:\n", r.stdout)
        print("STDERR:\n", r.stderr)
        raise RuntimeError(f"Vina failed rc={r.returncode} cfg={cfg_path}")
    return r

def parse_vina_scores(vina_out_pdbqt: Path):
    scores = []
    with Path(vina_out_pdbqt).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if ln.startswith("REMARK VINA RESULT:"):
                try:
                    scores.append(float(ln.split()[3]))
                except Exception:
                    pass
    return scores

# ---- MGLTools receptor prep ----
def prepare_receptor_pdbqt(pdb_in: Path, pdbqt_out: Path):
    cmd = [PYTHONSH, PREP_REC, "-r", str(pdb_in), "-o", str(pdbqt_out), "-A", "checkhydrogens"]
    r = subprocess.run(cmd, text=True, capture_output=True)
    if r.returncode != 0 or (not pdbqt_out.exists()) or pdbqt_out.stat().st_size < 200:
        print("STDOUT:\n", r.stdout)
        print("STDERR:\n", r.stderr)
        raise RuntimeError(f"prepare_receptor4.py failed for {pdb_in}")
    return pdbqt_out

# ---- PDBQT -> PDB line writer (Coot-safe element field) ----
def pdbqt_line_to_pdb(ln: str, serial: int, record="HETATM", chain="Z", resname="LIG", resi=1) -> str:
    ln = (ln.rstrip("\n") + " " * 80)[:80]
    atom_name = ln[12:16]
    altloc = ln[16:17]

    # coords
    x = ln[30:38]; y = ln[38:46]; z = ln[46:54]
    occ  = "  1.00"
    bfac = "  0.00"

    elem = _elem_from_pdbqt_line(atom_name, ln).rjust(2)[:2]

    # format like PDB (fixed columns)
    out = (
        f"{record.ljust(6)[:6]}{serial:5d} {atom_name}{altloc}"
        f"{resname[:3].upper():>3} {chain[:1]}{int(resi):4d}    "
        f"{x:>8}{y:>8}{z:>8}{occ}{bfac}          {elem}"
    )
    return out.rstrip() + "\n"

def extract_vina_model_atoms(vina_out_pdbqt: Path, model: int = 1):
    """
    Returns: (flex_atom_lines, ligand_atom_lines) from vina_out MODEL.
    flex_atom_lines: ATOM lines with AA3 resnames (protein)
    ligand_atom_lines: everything else ATOM/HETATM (ligand)
    """
    flex, lig = [], []
    cur_model = 0
    in_model = False
    saw_model = False

    with Path(vina_out_pdbqt).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if ln.startswith("MODEL"):
                saw_model = True
                cur_model += 1
                in_model = (cur_model == model)
                continue
            if ln.startswith("ENDMDL") and in_model:
                break
            if saw_model and (not in_model):
                continue

            if ln.startswith(("ATOM","HETATM")):
                resn = ln[17:20].strip().upper() if len(ln) >= 20 else ""
                if ln.startswith("ATOM") and resn in AA3:
                    flex.append(ln.rstrip("\n"))
                else:
                    lig.append(ln.rstrip("\n"))
    return flex, lig

def max_serial_in_pdb(pdb_path: Path) -> int:
    m = 0
    with Path(pdb_path).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if ln.startswith(("ATOM","HETATM")):
                try:
                    m = max(m, int(ln[6:11]))
                except Exception:
                    pass
    return m

# ---- Complex builders (rigid + flex) ----
def build_complex_rigid(
    receptor_pdb: Path,
    vina_out_pdbqt: Path,
    out_pdb: Path,
    model: int = 1,
    ligand_chain: str = "Z",
    ligand_resname: str = "LIG",
    ligand_resi: int = 1,
):
    receptor_pdb = Path(receptor_pdb)
    vina_out_pdbqt = Path(vina_out_pdbqt)
    out_pdb = Path(out_pdb)

    _, lig_raw = extract_vina_model_atoms(vina_out_pdbqt, model=model)
    if not lig_raw:
        raise RuntimeError(f"No ligand atoms parsed from {vina_out_pdbqt} (MODEL {model}).")

    serial = max_serial_in_pdb(receptor_pdb)

    with receptor_pdb.open("r", encoding="utf-8", errors="ignore") as f_in, out_pdb.open("w", encoding="utf-8") as f_out:
        for ln in f_in:
            if ln.startswith("END"):
                continue
            if ln.startswith(("ATOM","HETATM","TER")):
                f_out.write(ln if ln.endswith("\n") else ln + "\n")

        f_out.write("TER\n")

        # write ligand atoms (Coot-safe element + safe resname)
        for ln in lig_raw:
            serial += 1
            f_out.write(pdbqt_line_to_pdb(
                ln, serial,
                record="HETATM",
                chain=ligand_chain,
                resname=ligand_resname,
                resi=ligand_resi
            ))

        f_out.write("END\n")
    return out_pdb

def build_complex_flex(
    rigid_minus_flexatoms_pdb: Path,
    vina_out_pdbqt: Path,
    out_pdb: Path,
    model: int = 1,
    receptor_chain_for_flex: str = "C",
    ligand_chain: str = "Z",
    ligand_resname: str = "LIG",
    ligand_resi: int = 1,
):
    rigid_minus_flexatoms_pdb = Path(rigid_minus_flexatoms_pdb)
    vina_out_pdbqt = Path(vina_out_pdbqt)
    out_pdb = Path(out_pdb)

    flex_raw, lig_raw = extract_vina_model_atoms(vina_out_pdbqt, model=model)
    if not lig_raw:
        raise RuntimeError(f"No ligand atoms parsed from {vina_out_pdbqt} (MODEL {model}).")

    serial = max_serial_in_pdb(rigid_minus_flexatoms_pdb)

    with rigid_minus_flexatoms_pdb.open("r", encoding="utf-8", errors="ignore") as f_in, out_pdb.open("w", encoding="utf-8") as f_out:
        for ln in f_in:
            if ln.startswith("END"):
                continue
            f_out.write(ln if ln.endswith("\n") else ln + "\n")

        f_out.write("TER\n")

        # flex protein atoms
        for ln in flex_raw:
            serial += 1
            f_out.write(pdbqt_line_to_pdb(
                ln, serial,
                record="ATOM",
                chain=receptor_chain_for_flex,
                resname=ln[17:20].strip(),  # keep
                resi=int(ln[22:26])
            ))

        if flex_raw:
            f_out.write("TER\n")

        # ligand atoms
        for ln in lig_raw:
            serial += 1
            f_out.write(pdbqt_line_to_pdb(
                ln, serial,
                record="HETATM",
                chain=ligand_chain,
                resname=ligand_resname,
                resi=ligand_resi
            ))

        f_out.write("END\n")
    return out_pdb

print("OK: shared helpers loaded (Coot-safe ligand resname + element mapping).")


OK: shared helpers loaded (Coot-safe ligand resname + element mapping).


**Cell 04 — Project paths + inputs (your folders)**

In [4]:
from pathlib import Path
from datetime import datetime

# Run notebook from your project root:
PROJECT_ROOT = Path("/home/s94m696/projects/vinaflex").resolve()
assert PROJECT_ROOT.exists(), f"Missing PROJECT_ROOT: {PROJECT_ROOT}"
%cd $PROJECT_ROOT

# ---- Choose ONE of these ----
# Option A (manual tag - recommended so you recognize the run)
RUN_TAG = "run_20260129_v2"

# Option B (auto timestamp)
# RUN_TAG = datetime.now().strftime("run_%Y%m%d_%H%M%S")

# ---- New output root (won't overwrite old results) ----
OUT_ROOT = PROJECT_ROOT / "runs" / RUN_TAG
OUT_ROOT.mkdir(parents=True, exist_ok=True)

# Receptors
KEEP = [
    "8YIO_chain-C-minus-AZO-F129L-conf1.pdb",
    "8YIO_chain-C-minus-AZO-F129L-conf2.pdb",
    "8YIO_chain-C-minus-AZO-Gly137Arg-conf1.pdb",
    "8YIO_chain-C-minus-AZO-Gly137Arg-conf2.pdb",
    "8YIO_chain-C-minus-AZO-Gly143Ala.pdb",
    "8YIO_chain-C-minus-AZO.pdb",
]
RECEPTOR_PDBS = [str((PROJECT_ROOT / f).resolve()) for f in KEEP if (PROJECT_ROOT / f).exists()]
if not RECEPTOR_PDBS:
    RECEPTOR_PDBS = [str(p.resolve()) for p in sorted(PROJECT_ROOT.glob("8YIO_chain-C-minus-AZO*.pdb"))]
assert RECEPTOR_PDBS, "No receptor PDBs found."

# Ligands (PDBQT)
LIG_DIR = (PROJECT_ROOT / "ligands_vs_pdbqt").resolve()
assert LIG_DIR.exists(), f"Missing ligand folder: {LIG_DIR}"
LIGAND_PDBQTS = sorted([str(p.resolve()) for p in LIG_DIR.rglob("*.pdbqt") if "receptor" not in p.name.lower()])
assert LIGAND_PDBQTS, f"No ligand .pdbqt found under {LIG_DIR}"

# Outputs (NEW)
OUT_RIGID = OUT_ROOT / "dock_batch_rigid"
OUT_FLEX  = OUT_ROOT / "dock_batch_flex"
OUT_RIGID.mkdir(exist_ok=True, parents=True)
OUT_FLEX.mkdir(exist_ok=True, parents=True)

print("RUN_TAG :", RUN_TAG)
print("OUT_ROOT:", OUT_ROOT)
print("OUT_RIGID:", OUT_RIGID)
print("OUT_FLEX :", OUT_FLEX)

# expose to later cells
globals().update(dict(
    PROJECT_ROOT=PROJECT_ROOT,
    OUT_ROOT=OUT_ROOT,
    RUN_TAG=RUN_TAG,
    OUT_RIGID=OUT_RIGID,
    OUT_FLEX=OUT_FLEX,
    RECEPTOR_PDBS=RECEPTOR_PDBS,
    LIGAND_PDBQTS=LIGAND_PDBQTS,
))


/home/s94m696/projects/vinaflex
RUN_TAG : run_20260129_v2
OUT_ROOT: /home/s94m696/projects/vinaflex/runs/run_20260129_v2
OUT_RIGID: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_rigid
OUT_FLEX : /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex


**Cell 05 — Docking box (auto from crystal ligand + fallback)**

In [5]:
# Edit these if needed
AZO_CRYSTAL_PDB    = str((PROJECT_ROOT / "AZO_crystal.pdb").resolve())  # ligand coordinates from crystal
WT_RECEPTOR_PDB    = str((PROJECT_ROOT / "8YIO_chain-C-minus-AZO.pdb").resolve())
BOX_SIZE           = 22.0
FALLBACK_CENTER_WT = (3.692, -1.725, -13.680)

assert Path(AZO_CRYSTAL_PDB).exists(), f"Missing: {AZO_CRYSTAL_PDB}"
assert Path(WT_RECEPTOR_PDB).exists(), f"Missing: {WT_RECEPTOR_PDB}"

xs, ys, zs = read_xyz_from_pdb(AZO_CRYSTAL_PDB)
center_auto = (float(xs.mean()), float(ys.mean()), float(zs.mean()))
size = (float(BOX_SIZE), float(BOX_SIZE), float(BOX_SIZE))

inside_auto = qc_box_intersection_pdb(WT_RECEPTOR_PDB, center_auto, size)
if inside_auto == 0:
    center = tuple(map(float, FALLBACK_CENTER_WT))
else:
    center = center_auto

box = dict(center=center, size=size)
print("Box center:", box["center"])
print("Box size  :", box["size"])
print("WT atoms inside box:", qc_box_intersection_pdb(WT_RECEPTOR_PDB, box["center"], box["size"]))


Box center: (3.692, -1.725, -13.68)
Box size  : (22.0, 22.0, 22.0)
WT atoms inside box: 417


**Cell 06 — Batch rigid docking (mutants × ligands) + build complexes (Coot-safe) + summary**

In [6]:
EXHAUSTIVENESS_RIGID = 24
NUM_MODES_RIGID      = 20
ENERGY_RANGE_RIGID   = 4
SEED_RIGID           = 42

REC_CACHE = OUT_RIGID / "_receptor_cache"
REC_CACHE.mkdir(exist_ok=True, parents=True)

def get_cached_receptor_pdbqt(mut_pdb: str) -> Path:
    mut_pdb = Path(mut_pdb)
    d = REC_CACHE / mut_pdb.stem
    d.mkdir(exist_ok=True, parents=True)
    rec_pdbqt = d / "receptor.pdbqt"
    if (not rec_pdbqt.exists()) or rec_pdbqt.stat().st_size < 200:
        prepare_receptor_pdbqt(mut_pdb, rec_pdbqt)
    return rec_pdbqt

rows = []
center = tuple(map(float, box["center"]))
size   = tuple(map(float, box["size"]))

for lig in LIGAND_PDBQTS:
    lig = str(lig)
    lig_tag = Path(lig).stem
    lig_resname = safe_lig_resname_from_path(lig)  # IMPORTANT FIX (no MET)
    for mut in RECEPTOR_PDBS:
        mut = str(mut)
        mut_tag = Path(mut).stem

        outdir = OUT_RIGID / lig_tag / mut_tag
        outdir.mkdir(exist_ok=True, parents=True)

        rec_pdbqt = get_cached_receptor_pdbqt(mut)
        cfg       = outdir / "vina.cfg"
        logp      = outdir / "vina.log"
        vina_out  = outdir / "vina_out.pdbqt"
        complex_pdb = outdir / "complex_top1.pdb"

        # run docking if missing
        if (not vina_out.exists()) or vina_out.stat().st_size < 500:
            write_vina_cfg_rigid(
                cfg_path=cfg,
                receptor_pdbqt=rec_pdbqt,
                ligand_pdbqt=Path(lig),
                center=center,
                size=size,
                out_pdbqt=vina_out,
                exhaustiveness=EXHAUSTIVENESS_RIGID,
                num_modes=NUM_MODES_RIGID,
                energy_range=ENERGY_RANGE_RIGID,
                seed=SEED_RIGID,
                cpu=CPU_TO_USE,
                log_path=logp
            )
            print(f"[RIGID VINA] {lig_tag} | {mut_tag}")
            run_vina(cfg)

        scores = parse_vina_scores(vina_out)
        best = scores[0] if scores else float("nan")

        # build complex (Coot-safe)
        build_complex_rigid(
            receptor_pdb=Path(mut),
            vina_out_pdbqt=vina_out,
            out_pdb=complex_pdb,
            model=1,
            ligand_chain="Z",
            ligand_resname=lig_resname,
            ligand_resi=1,
        )

        rows.append({
            "ligand": lig_tag,
            "lig_resname": lig_resname,
            "mutant": mut_tag,
            "best_kcal_mol": best,
            "vina_out": str(vina_out),
            "complex_top1": str(complex_pdb),
        })

df_rigid = pd.DataFrame(rows)
df_rigid.to_csv(OUT_RIGID / "summary_rigid.csv", index=False)
print("Saved:", (OUT_RIGID / "summary_rigid.csv").resolve())
df_rigid.head()


Saved: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_rigid/summary_rigid.csv


,ligand,lig_resname,mutant,best_kcal_mol,vina_out,complex_top1
0,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-F129L-conf1,-9.7,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
1,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-F129L-conf2,-9.6,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
2,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly137Arg-conf1,-9.9,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
3,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly137Arg-conf2,-9.9,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
4,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly143Ala,-8.7,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...


# Flexible docking section (requires MGLTools)

Only run the next cells with:
- `pythonsh` (MGLTools)
- `prepare_flexreceptor4.py` available

**Cell 07 — Flex sites (custom list)**

In [7]:
RECEPTOR_CHAIN = "C"
FLEX_SITES = [
    (RECEPTOR_CHAIN, 125),
    (RECEPTOR_CHAIN, 128),
    (RECEPTOR_CHAIN, 129),
    (RECEPTOR_CHAIN, 132),
    (RECEPTOR_CHAIN, 140),
    (RECEPTOR_CHAIN, 143),
    (RECEPTOR_CHAIN, 144),
    (RECEPTOR_CHAIN, 147),
    (RECEPTOR_CHAIN, 270),
    (RECEPTOR_CHAIN, 271),
    (RECEPTOR_CHAIN, 272),
    (RECEPTOR_CHAIN, 278),
]
print("FLEX_SITES:", FLEX_SITES)


FLEX_SITES: [('C', 125), ('C', 128), ('C', 129), ('C', 132), ('C', 140), ('C', 143), ('C', 144), ('C', 147), ('C', 270), ('C', 271), ('C', 272), ('C', 278)]


**Cell 08 — Flex receptor cache prep (full→rigid+flex PDBQT + rigid-minus-flexatoms PDB)**

In [8]:
OUT_FLEX.mkdir(exist_ok=True, parents=True)
FLEX_CACHE = OUT_FLEX / "_receptor_cache"
FLEX_CACHE.mkdir(exist_ok=True, parents=True)

MIN_FULL_PDBQT_BYTES  = 500
MIN_RIGID_PDBQT_BYTES = 200
MIN_FLEX_PDBQT_BYTES  = 200

def _file_small(p: Path, min_bytes: int) -> bool:
    return (not p.exists()) or (p.stat().st_size < min_bytes)

def _count_atom_lines(p: Path) -> int:
    if not Path(p).exists():
        return 0
    return sum(1 for ln in Path(p).open("r", errors="ignore") if ln.startswith("ATOM"))

def parse_resname_map_from_pdbqt(pdbqt_path: Path) -> dict:
    m = {}
    with Path(pdbqt_path).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if not ln.startswith("ATOM") or len(ln) < 26:
                continue
            resn = ln[17:20].strip().upper()
            ch = (ln[21].strip() or " ")
            try:
                resi = int(ln[22:26])
            except Exception:
                continue
            if resn in AA3:
                m[(ch, resi)] = resn
    return m

def make_flex_spec_from_sites(full_receptor_pdbqt: Path, flex_sites) -> str:
    mol = Path(full_receptor_pdbqt).stem  # receptor_full
    resmap = parse_resname_map_from_pdbqt(full_receptor_pdbqt)
    selectors = []
    skipped = []
    for ch, resi in flex_sites:
        ch2 = (str(ch).strip() or " ")
        key = (ch2, int(resi))
        resn = resmap.get(key)
        if not resn:
            skipped.append(key)
            continue
        selectors.append(f"{mol}:{ch2}:{resn}{int(resi)}")
    if skipped:
        print("WARNING: skipped FLEX_SITES not found in this receptor:", skipped)
    if not selectors:
        raise RuntimeError("No valid flexible residues found (selector empty).")
    return ",".join(selectors)

def prepare_flex_split(full_receptor_pdbqt: Path, rigid_out: Path, flex_out: Path, flex_sites):
    flex_spec = make_flex_spec_from_sites(full_receptor_pdbqt, flex_sites)
    if rigid_out.exists(): rigid_out.unlink()
    if flex_out.exists():  flex_out.unlink()

    cmd = [
        PYTHONSH, PREP_FLEX,
        "-r", str(full_receptor_pdbqt),
        "-s", flex_spec,
        "-g", str(rigid_out),
        "-x", str(flex_out),
    ]
    r = subprocess.run(cmd, text=True, capture_output=True)
    if r.returncode != 0:
        print("STDOUT:\n", r.stdout)
        print("STDERR:\n", r.stderr)
        raise RuntimeError("prepare_flexreceptor4.py failed.")
    if _count_atom_lines(flex_out) == 0:
        raise RuntimeError(f"Flex PDBQT has 0 ATOM lines: {flex_out}")

def parse_flex_atoms_from_pdbqt(flex_pdbqt: Path) -> set:
    s = set()
    with Path(flex_pdbqt).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if not ln.startswith("ATOM") or len(ln) < 26:
                continue
            atom = ln[12:16].strip()
            ch = (ln[21].strip() or " ")
            try:
                resi = int(ln[22:26])
            except Exception:
                continue
            resn = ln[17:20].strip().upper()
            if resn in AA3:
                s.add((ch, resi, atom))
    return s

def strip_flex_atoms_from_pdb(pdb_in: Path, pdb_out: Path, flex_atoms: set):
    kept_any = False
    with Path(pdb_in).open("r", encoding="utf-8", errors="ignore") as f_in, Path(pdb_out).open("w", encoding="utf-8") as f_out:
        for ln in f_in:
            if ln.startswith("ATOM") and len(ln) >= 26:
                atom = ln[12:16].strip()
                ch = (ln[21].strip() or " ")
                try:
                    resi = int(ln[22:26])
                except Exception:
                    f_out.write(ln if ln.endswith("\n") else ln + "\n")
                    kept_any = True
                    continue
                if (ch, resi, atom) in flex_atoms:
                    continue
            f_out.write(ln if ln.endswith("\n") else ln + "\n")
            if ln.startswith(("ATOM","HETATM")):
                kept_any = True
    if not kept_any:
        raise RuntimeError(f"strip_flex_atoms_from_pdb produced empty PDB: {pdb_out}")

def get_flex_prepped_files(mut_pdb: str):
    mut_pdb = Path(mut_pdb)
    tag = mut_pdb.stem
    d = FLEX_CACHE / tag
    d.mkdir(exist_ok=True, parents=True)

    full_pdbqt  = d / "receptor_full.pdbqt"
    rigid_pdbqt = d / "receptor_rigid.pdbqt"
    flex_pdbqt  = d / "receptor_flex.pdbqt"
    rigid_minus = d / "receptor_rigid_minus_flexatoms.pdb"

    if _file_small(full_pdbqt, MIN_FULL_PDBQT_BYTES):
        print(f"[prep full receptor] {tag}")
        prepare_receptor_pdbqt(mut_pdb, full_pdbqt)

    if _file_small(rigid_pdbqt, MIN_RIGID_PDBQT_BYTES) or _file_small(flex_pdbqt, MIN_FLEX_PDBQT_BYTES):
        print(f"[prep flex split] {tag}")
        prepare_flex_split(full_pdbqt, rigid_pdbqt, flex_pdbqt, FLEX_SITES)

    if _file_small(rigid_minus, 500):
        print(f"[prep rigid-minus-flexatoms PDB] {tag}")
        flex_atoms = parse_flex_atoms_from_pdbqt(flex_pdbqt)
        strip_flex_atoms_from_pdb(mut_pdb, rigid_minus, flex_atoms)

    if _count_atom_lines(flex_pdbqt) == 0:
        raise RuntimeError(f"Flex file ended up empty: {flex_pdbqt}")

    return full_pdbqt, rigid_pdbqt, flex_pdbqt, rigid_minus

print("OK: get_flex_prepped_files() ready")


OK: get_flex_prepped_files() ready


**Cell 09 — Batch flex docking + build complexes (Coot-safe) + summary**

In [9]:
EXHAUSTIVENESS_FLEX = 24
NUM_MODES_FLEX      = 20
ENERGY_RANGE_FLEX   = 4
SEED_FLEX           = 42

MIN_VINA_OUT_BYTES = 500

def looks_ok_vina_out(p: Path) -> bool:
    return p.exists() and p.stat().st_size >= MIN_VINA_OUT_BYTES

def count_flex_atoms_in_vina_out_model1(vina_out_pdbqt: Path) -> int:
    n = 0
    in_model = False
    saw_model = False
    with Path(vina_out_pdbqt).open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if ln.startswith("MODEL"):
                saw_model = True
                in_model = True
                continue
            if ln.startswith("ENDMDL") and in_model:
                break
            if saw_model and (not in_model):
                continue
            if ln.startswith("ATOM"):
                resn = ln[17:20].strip().upper() if len(ln) >= 20 else ""
                if resn in AA3:
                    n += 1
    return n

def needs_rebuild(complex_pdb: Path, vina_out: Path) -> bool:
    if (not complex_pdb.exists()) or (complex_pdb.stat().st_size < 500):
        return True
    try:
        return complex_pdb.stat().st_mtime < vina_out.stat().st_mtime
    except Exception:
        return True

rows = []
center = tuple(map(float, box["center"]))
size   = tuple(map(float, box["size"]))

for lig in LIGAND_PDBQTS:
    lig = str(lig)
    lig_tag = Path(lig).stem
    lig_resname = safe_lig_resname_from_path(lig)  # IMPORTANT FIX (no MET)

    for mut in RECEPTOR_PDBS:
        mut = str(mut)
        mut_tag = Path(mut).stem

        outdir = OUT_FLEX / lig_tag / mut_tag
        outdir.mkdir(exist_ok=True, parents=True)

        cfg       = outdir / "vina.cfg"
        vina_log  = outdir / "vina.log"
        vina_out  = outdir / "vina_out.pdbqt"
        complex_pdb = outdir / "complex_top1.pdb"

        full_rec, rigid_rec, flex_rec, rigid_minus = get_flex_prepped_files(mut)

        need_vina = (not looks_ok_vina_out(vina_out)) or (count_flex_atoms_in_vina_out_model1(vina_out) < 1)
        if need_vina:
            write_vina_cfg_flex(
                cfg_path=cfg,
                receptor_pdbqt=rigid_rec,
                flex_pdbqt=flex_rec,
                ligand_pdbqt=Path(lig),
                center=center,
                size=size,
                out_pdbqt=vina_out,
                log_path=vina_log,
                exhaustiveness=EXHAUSTIVENESS_FLEX,
                num_modes=NUM_MODES_FLEX,
                energy_range=ENERGY_RANGE_FLEX,
                seed=SEED_FLEX,
                cpu=CPU_TO_USE
            )
            print(f"[FLEX VINA] {lig_tag} | {mut_tag}")
            run_vina(cfg)

        if count_flex_atoms_in_vina_out_model1(vina_out) < 1:
            raise RuntimeError(f"QC FAIL: no flex atoms in MODEL1 for {vina_out}")

        if needs_rebuild(complex_pdb, vina_out):
            build_complex_flex(
                rigid_minus_flexatoms_pdb=rigid_minus,
                vina_out_pdbqt=vina_out,
                out_pdb=complex_pdb,
                model=1,
                receptor_chain_for_flex="C",
                ligand_chain="Z",
                ligand_resname=lig_resname,
                ligand_resi=1
            )

        scores = parse_vina_scores(vina_out)
        best = scores[0] if scores else float("nan")

        rows.append({
            "ligand": lig_tag,
            "lig_resname": lig_resname,
            "mutant": mut_tag,
            "best_kcal_mol": best,
            "vina_out": str(vina_out),
            "vina_log": str(vina_log),
            "complex_top1": str(complex_pdb),
        })

df_flex = pd.DataFrame(rows)
df_flex.to_csv(OUT_FLEX / "summary_flex.csv", index=False)
print("Saved:", (OUT_FLEX / "summary_flex.csv").resolve())
df_flex.head()


Saved: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/summary_flex.csv


,ligand,lig_resname,mutant,best_kcal_mol,vina_out,vina_log,complex_top1
0,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-F129L-conf1,-10.0,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
1,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-F129L-conf2,-9.7,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
2,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly137Arg-conf1,-10.1,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
3,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly137Arg-conf2,-10.0,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...
4,azoxystrobin_mol1,AZO,8YIO_chain-C-minus-AZO-Gly143Ala,-8.6,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...,/home/s94m696/projects/vinaflex/runs/run_20260...


# Docking completion report. To check if docking is completed. #

In [10]:
from pathlib import Path
import pandas as pd

RUN_DIR = Path("/home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex").resolve()
assert RUN_DIR.exists(), f"Not found: {RUN_DIR}"

AA3 = {
    "ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE",
    "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL",
    "HSD","HSE","HSP","MSE"
}

def count_vina_scores(vina_out: Path) -> int:
    n = 0
    with vina_out.open("r", errors="ignore") as f:
        for ln in f:
            if ln.startswith("REMARK VINA RESULT:"):
                n += 1
    return n

def count_flex_atoms_model1(vina_out: Path) -> int:
    """
    In flex docking, vina_out.pdbqt contains protein ATOM lines for flexible residues.
    Count AA3 ATOM lines in MODEL 1 (or whole file if no MODEL).
    """
    n = 0
    saw_model = False
    in_model1 = True
    with vina_out.open("r", errors="ignore") as f:
        for ln in f:
            if ln.startswith("MODEL"):
                saw_model = True
                in_model1 = True  # first model starts
                continue
            if saw_model and ln.startswith("ENDMDL"):
                break
            if ln.startswith("ATOM"):
                resn = ln[17:20].strip().upper() if len(ln) >= 20 else ""
                if resn in AA3:
                    n += 1
    return n

def is_complex_fresh(complex_pdb: Path, vina_out: Path) -> bool:
    if (not complex_pdb.exists()) or (complex_pdb.stat().st_size < 500):
        return False
    try:
        return complex_pdb.stat().st_mtime >= vina_out.stat().st_mtime
    except Exception:
        return False

# Infer pairs from folder structure: RUN_DIR/<lig>/<mut>/
lig_dirs = sorted([p for p in RUN_DIR.iterdir() if p.is_dir() and not p.name.startswith("_")])
rows = []

for lig_dir in lig_dirs:
    mut_dirs = sorted([p for p in lig_dir.iterdir() if p.is_dir() and not p.name.startswith("_")])
    for mut_dir in mut_dirs:
        vina_out = mut_dir / "vina_out.pdbqt"
        vina_log = mut_dir / "vina.log"
        complex_pdb = mut_dir / "complex_top1.pdb"

        vina_out_exists = vina_out.exists() and vina_out.stat().st_size > 200
        scores_n = count_vina_scores(vina_out) if vina_out_exists else 0
        flex_atoms_n = count_flex_atoms_model1(vina_out) if vina_out_exists else 0
        complex_ok = vina_out_exists and is_complex_fresh(complex_pdb, vina_out)

        # “Done” definition for FLEX:
        done = vina_out_exists and (scores_n > 0) and (flex_atoms_n > 0) and complex_ok

        rows.append({
            "ligand": lig_dir.name,
            "mutant": mut_dir.name,
            "outdir": str(mut_dir),
            "vina_out_exists": vina_out_exists,
            "scores_n": int(scores_n),
            "flex_atoms_model1_n": int(flex_atoms_n),
            "complex_exists": complex_pdb.exists(),
            "complex_fresh_vs_vina_out": bool(complex_ok),
            "DONE": bool(done),
            "vina_out": str(vina_out),
            "vina_log": str(vina_log),
            "complex_top1": str(complex_pdb),
        })

df = pd.DataFrame(rows)
print("RUN_DIR:", RUN_DIR)
print("Total folders (lig×mut) found:", len(df))
print("DONE:", int(df["DONE"].sum()), "/", len(df))

# show the incomplete ones (most useful)
incomplete = df[~df["DONE"]].copy()
display(incomplete[["ligand","mutant","vina_out_exists","scores_n","flex_atoms_model1_n","complex_exists","complex_fresh_vs_vina_out","outdir"]].head(50))

# Save report
report_csv = RUN_DIR / "docking_completion_report.csv"
df.to_csv(report_csv, index=False)
print("Wrote:", report_csv)

# If you want a clean “missing list”:
missing_list = incomplete[["ligand","mutant","outdir"]]
missing_csv = RUN_DIR / "docking_missing.csv"
missing_list.to_csv(missing_csv, index=False)
print("Wrote:", missing_csv)


RUN_DIR: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex
Total folders (lig×mut) found: 126
DONE: 126 / 126


,ligand,mutant,vina_out_exists,scores_n,flex_atoms_model1_n,complex_exists,complex_fresh_vs_vina_out,outdir


Wrote: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/docking_completion_report.csv
Wrote: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/docking_missing.csv


**Cell 10 — Rebuild complexes ONLY (no re-docking), use this to fix existing “weird in Coot” files**
*annoyingly, two ligands i.e. metylterapole and metominostrobin look weird in the complex (look ok if visualized using PyMol). Coot is more finicky than PyMol or Chimera*

In [11]:
rebuilt = 0
vina_out_files = sorted([p for p in OUT_FLEX.glob("*/*/vina_out.pdbqt") if "_receptor_cache" not in str(p)])

tag2pdb = {Path(p).stem: str(Path(p).resolve()) for p in RECEPTOR_PDBS}

for vina_out in vina_out_files:
    lig_tag = vina_out.parents[1].name
    mut_tag = vina_out.parents[0].name
    outdir  = vina_out.parent

    complex_pdb = outdir / "complex_top1.pdb"

    mut_pdb = tag2pdb.get(mut_tag)
    if not mut_pdb:
        continue

    # find ligand pdbqt to derive safe resname
    # (we just use folder name as tag; safe mapping still works)
    lig_resname = safe_lig_resname_from_path(lig_tag)

    _full, _rigid, _flex, rigid_minus = get_flex_prepped_files(mut_pdb)

    build_complex_flex(
        rigid_minus_flexatoms_pdb=rigid_minus,
        vina_out_pdbqt=vina_out,
        out_pdb=complex_pdb,
        model=1,
        receptor_chain_for_flex="C",
        ligand_chain="Z",
        ligand_resname=lig_resname,
        ligand_resi=1
    )
    rebuilt += 1

print("Rebuilt complexes:", rebuilt)


Rebuilt complexes: 126


**Cell 11 — Weak-force parsing + contact computation (shared)**
*WEAK-FORCE / FEATURES (for BOTH rigid and flex)*

In [12]:
NONPOLAR = {"C","S"}

def parse_pdb_atoms_df(pdb_path: Path) -> pd.DataFrame:
    rows = []
    pdb_path = Path(pdb_path)
    with pdb_path.open("r", encoding="utf-8", errors="ignore") as f:
        for ln in f:
            if not (ln.startswith("ATOM") or ln.startswith("HETATM")):
                continue
            if len(ln) < 54:
                continue
            ln = ln.rstrip("\n")
            if len(ln) < 80:
                ln = ln.ljust(80)

            rec = ln[0:6].strip()
            try:
                serial = int(ln[6:11])
            except Exception:
                serial = None
            atom = ln[12:16].strip()
            resname = ln[17:20].strip().upper()
            chain = (ln[21:22].strip() or " ")
            try:
                resi = int(ln[22:26])
            except Exception:
                continue
            try:
                x = float(ln[30:38]); y = float(ln[38:46]); z = float(ln[46:54])
            except Exception:
                continue

            elem_field = ln[76:78].strip().upper()
            element = elem_field if elem_field else _infer_element_from_atom(atom)
            if resname in WAT:
                continue
            if element == "H" or atom.upper().startswith("H"):
                continue

            rows.append((rec, serial, atom, resname, chain, resi, x, y, z, element))

    df = pd.DataFrame(rows, columns=["record","serial","atom","resname","chain","resi","x","y","z","element"])
    if df.empty:
        raise RuntimeError(f"No ATOM/HETATM parsed from: {pdb_path}")
    return df

def split_rec_lig(df: pd.DataFrame, receptor_chain="C", ligand_chain="Z"):
    rec = df[(df["record"] == "ATOM") & (df["chain"] == receptor_chain) & (df["resname"].isin(AA3))].copy()
    lig = df[(df["chain"] == ligand_chain) & (~df["resname"].isin(AA3))].copy()
    if lig.empty:
        lig = df[(df["record"] == "HETATM") & (~df["resname"].isin(AA3))].copy()
    return rec, lig

def compute_contacts_df(rec: pd.DataFrame, lig: pd.DataFrame, cutoff_A=4.5) -> pd.DataFrame:
    if rec.empty or lig.empty:
        return pd.DataFrame()

    R = rec[["x","y","z"]].to_numpy(np.float64)
    L = lig[["x","y","z"]].to_numpy(np.float64)
    cutoff2 = float(cutoff_A) ** 2

    out = []
    chunk = 64
    for i0 in range(0, L.shape[0], chunk):
        Lc = L[i0:i0+chunk]
        d = Lc[:, None, :] - R[None, :, :]
        d2 = np.sum(d*d, axis=2)
        li, ri = np.where(d2 <= cutoff2)
        for a, b in zip(li, ri):
            lig_idx = i0 + a
            dist = float(math.sqrt(d2[a, b]))
            rr = rec.iloc[b]
            ll = lig.iloc[lig_idx]
            out.append({
                "dist_A": dist,

                "rec_chain": rr["chain"],
                "rec_resname": rr["resname"],
                "rec_resi": int(rr["resi"]),
                "rec_atom": rr["atom"],
                "rec_element": rr["element"],
                "rec_serial": rr["serial"],

                "lig_chain": ll["chain"],
                "lig_resname": ll["resname"],
                "lig_resi": int(ll["resi"]),
                "lig_atom": ll["atom"],
                "lig_element": ll["element"],
                "lig_serial": ll["serial"],
            })

    if not out:
        return pd.DataFrame()
    return pd.DataFrame(out).sort_values("dist_A").reset_index(drop=True)

def classify_weak_force(row) -> str:
    d = float(row["dist_A"])
    re_ = str(row["rec_element"]).upper()
    le_ = str(row["lig_element"]).upper()

    if re_ in METALS or le_ in METALS:
        return "metal_coordination_like" if d <= 2.8 else "metal_proximity"

    if (re_ in NONPOLAR) and (le_ in NONPOLAR) and d <= 4.0:
        return "hydrophobic_vdw"

    if le_ in HALOGENS and re_ in {"O","N","S"} and d <= 3.6:
        return "halogen_bond_like"
    if re_ in HALOGENS and le_ in {"O","N","S"} and d <= 3.6:
        return "halogen_bond_like"

    if re_ in {"N","O","S"} and le_ in {"N","O","S"} and d <= 3.5:
        return "hbond_like"

    if ((re_ == "N" and le_ == "O") or (re_ == "O" and le_ == "N")) and d <= 4.0:
        return "electrostatic_like"

    return "polar_or_vdw" if d <= 4.5 else "none"

def summarize_by_residue(dfc: pd.DataFrame) -> pd.DataFrame:
    if dfc.empty:
        return pd.DataFrame()
    df = dfc.copy()
    df["interaction"] = df.apply(classify_weak_force, axis=1)
    g = df.groupby(["rec_chain","rec_resname","rec_resi","interaction"], dropna=False)
    out = g.agg(n_contacts=("dist_A","size"), min_dist_A=("dist_A","min")).reset_index()
    return out.sort_values(["min_dist_A","n_contacts"], ascending=[True, False]).reset_index(drop=True)

print("OK: weak-force/contact helpers ready")


OK: weak-force/contact helpers ready


**Cell 12 — Weak-force batch for RIGID complexes**

In [13]:
ANALYSIS_DIR = OUT_RIGID / "_analysis"
ATOM_DIR = ANALYSIS_DIR / "contacts_atoms"
RES_DIR  = ANALYSIS_DIR / "contacts_residues"
ATOM_DIR.mkdir(parents=True, exist_ok=True)
RES_DIR.mkdir(parents=True, exist_ok=True)

complex_files = sorted(OUT_RIGID.glob("*/*/complex_top1.pdb"))
print("Rigid complexes found:", len(complex_files))

feature_rows = []
for cpdb in complex_files:
    lig_tag = cpdb.parent.parent.name
    mut_tag = cpdb.parent.name
    tag = f"{lig_tag}__{mut_tag}"

    atoms = parse_pdb_atoms_df(cpdb)
    rec, lig = split_rec_lig(atoms, receptor_chain="C", ligand_chain="Z")
    contacts = compute_contacts_df(rec, lig, cutoff_A=4.5)

    if contacts.empty:
        feature_rows.append({
            "ligand": lig_tag, "mutant": mut_tag, "complex_pdb": str(cpdb),
            "n_contacts_4p5A": 0,
            "n_hbond_like": 0,
            "n_electrostatic_like": 0,
            "n_hydrophobic_vdw": 0,
            "n_halogen_bond_like": 0,
            "n_metal_coord_like": 0,
        })
        continue

    contacts["interaction"] = contacts.apply(classify_weak_force, axis=1)
    contacts.to_csv(ATOM_DIR / f"{tag}.csv", index=False)

    res_sum = summarize_by_residue(contacts)
    res_sum.to_csv(RES_DIR / f"{tag}.csv", index=False)

    counts = contacts["interaction"].value_counts()
    feature_rows.append({
        "ligand": lig_tag, "mutant": mut_tag, "complex_pdb": str(cpdb),
        "n_contacts_4p5A": int(len(contacts)),
        "n_hbond_like": int(counts.get("hbond_like", 0)),
        "n_electrostatic_like": int(counts.get("electrostatic_like", 0)),
        "n_hydrophobic_vdw": int(counts.get("hydrophobic_vdw", 0)),
        "n_halogen_bond_like": int(counts.get("halogen_bond_like", 0)),
        "n_metal_coord_like": int(counts.get("metal_coordination_like", 0)),
    })

features_df = pd.DataFrame(feature_rows)
features_csv = ANALYSIS_DIR / "summary_features.csv"
features_df.to_csv(features_csv, index=False)
print("Wrote:", features_csv)
features_df.head()


Rigid complexes found: 126
Wrote: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_rigid/_analysis/summary_features.csv


,ligand,mutant,complex_pdb,n_contacts_4p5A,n_hbond_like,n_electrostatic_like,n_hydrophobic_vdw,n_halogen_bond_like,n_metal_coord_like
0,azoxystrobin_mol1,8YIO_chain-C-minus-AZO,/home/s94m696/projects/vinaflex/runs/run_20260...,202,1,1,51,0,0
1,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-F129L-conf1,/home/s94m696/projects/vinaflex/runs/run_20260...,201,1,1,53,0,0
2,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-F129L-conf2,/home/s94m696/projects/vinaflex/runs/run_20260...,195,1,1,55,0,0
3,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-Gly137Arg-conf1,/home/s94m696/projects/vinaflex/runs/run_20260...,201,1,1,50,0,0
4,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-Gly137Arg-conf2,/home/s94m696/projects/vinaflex/runs/run_20260...,201,1,1,51,0,0


**Cell 13 — Weak-force batch for FLEX complexes**

In [14]:
ANALYSIS_DIR = OUT_FLEX / "_analysis"
ATOM_DIR = ANALYSIS_DIR / "contacts_atoms"
RES_DIR  = ANALYSIS_DIR / "contacts_residues"
ATOM_DIR.mkdir(parents=True, exist_ok=True)
RES_DIR.mkdir(parents=True, exist_ok=True)

complex_files = sorted(OUT_FLEX.glob("*/*/complex_top1.pdb"))
print("Flex complexes found:", len(complex_files))

feature_rows = []
for cpdb in complex_files:
    lig_tag = cpdb.parent.parent.name
    mut_tag = cpdb.parent.name
    tag = f"{lig_tag}__{mut_tag}"

    atoms = parse_pdb_atoms_df(cpdb)
    rec, lig = split_rec_lig(atoms, receptor_chain="C", ligand_chain="Z")
    contacts = compute_contacts_df(rec, lig, cutoff_A=4.5)

    if contacts.empty:
        feature_rows.append({
            "ligand": lig_tag, "mutant": mut_tag, "complex_pdb": str(cpdb),
            "n_contacts_4p5A": 0,
            "n_hbond_like": 0,
            "n_electrostatic_like": 0,
            "n_hydrophobic_vdw": 0,
            "n_halogen_bond_like": 0,
            "n_metal_coord_like": 0,
        })
        continue

    contacts["interaction"] = contacts.apply(classify_weak_force, axis=1)
    contacts.to_csv(ATOM_DIR / f"{tag}.csv", index=False)

    res_sum = summarize_by_residue(contacts)
    res_sum.to_csv(RES_DIR / f"{tag}.csv", index=False)

    counts = contacts["interaction"].value_counts()
    feature_rows.append({
        "ligand": lig_tag, "mutant": mut_tag, "complex_pdb": str(cpdb),
        "n_contacts_4p5A": int(len(contacts)),
        "n_hbond_like": int(counts.get("hbond_like", 0)),
        "n_electrostatic_like": int(counts.get("electrostatic_like", 0)),
        "n_hydrophobic_vdw": int(counts.get("hydrophobic_vdw", 0)),
        "n_halogen_bond_like": int(counts.get("halogen_bond_like", 0)),
        "n_metal_coord_like": int(counts.get("metal_coordination_like", 0)),
    })

features_df = pd.DataFrame(feature_rows)
features_csv = ANALYSIS_DIR / "summary_features.csv"
features_df.to_csv(features_csv, index=False)
print("Wrote:", features_csv)
features_df.head()


Flex complexes found: 126
Wrote: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/_analysis/summary_features.csv


,ligand,mutant,complex_pdb,n_contacts_4p5A,n_hbond_like,n_electrostatic_like,n_hydrophobic_vdw,n_halogen_bond_like,n_metal_coord_like
0,azoxystrobin_mol1,8YIO_chain-C-minus-AZO,/home/s94m696/projects/vinaflex/runs/run_20260...,204,1,0,56,0,0
1,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-F129L-conf1,/home/s94m696/projects/vinaflex/runs/run_20260...,207,1,2,59,0,0
2,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-F129L-conf2,/home/s94m696/projects/vinaflex/runs/run_20260...,203,2,1,58,0,0
3,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-Gly137Arg-conf1,/home/s94m696/projects/vinaflex/runs/run_20260...,205,1,0,56,0,0
4,azoxystrobin_mol1,8YIO_chain-C-minus-AZO-Gly137Arg-conf2,/home/s94m696/projects/vinaflex/runs/run_20260...,207,1,0,55,0,0


**Cell 14 — Sanity checks specifically for the Coot issue (element + resname)**

In [15]:
# Check that no ligand atoms have element "A" in a few complexes
import random

def grep_bad_element_A(pdb_path: Path, max_hits=5):
    hits = []
    for ln in Path(pdb_path).open(errors="ignore"):
        if ln.startswith("HETATM") and len(ln) >= 78:
            elem = ln[76:78].strip()
            if elem == "A":
                hits.append(ln.rstrip("\n"))
                if len(hits) >= max_hits:
                    break
    return hits

samples = random.sample(sorted((OUT_FLEX.glob("*/*/complex_top1.pdb"))), k=min(5, len(list(OUT_FLEX.glob("*/*/complex_top1.pdb")))))
for f in samples:
    bad = grep_bad_element_A(f)
    print("\nFile:", f)
    print("Bad element 'A' lines:", len(bad))
    for ln in bad:
        print(" ", ln)

# Confirm ligand resname is NOT an amino acid (MET etc.)
def ligand_resnames_in_complex(pdb_path: Path, lig_chain="Z"):
    s = set()
    for ln in Path(pdb_path).open(errors="ignore"):
        if ln.startswith("HETATM") and len(ln) >= 22 and ln[21] == lig_chain:
            s.add(ln[17:20].strip().upper())
    return sorted(s)

if samples:
    f = samples[0]
    print("\nExample ligand resnames in", f.name, ":", ligand_resnames_in_complex(f, "Z"))



File: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/azoxystrobin_mol1/8YIO_chain-C-minus-AZO-Gly137Arg-conf2/complex_top1.pdb
Bad element 'A' lines: 0

File: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/fenamidone_mol1/8YIO_chain-C-minus-AZO/complex_top1.pdb
Bad element 'A' lines: 0

File: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/orysastrobin_mol1/8YIO_chain-C-minus-AZO-Gly143Ala/complex_top1.pdb
Bad element 'A' lines: 0

File: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/fenaminstrobin_mol1/8YIO_chain-C-minus-AZO-Gly137Arg-conf2/complex_top1.pdb
Bad element 'A' lines: 0

File: /home/s94m696/projects/vinaflex/runs/run_20260129_v2/dock_batch_flex/pyraclostrobin_mol1/8YIO_chain-C-minus-AZO-Gly137Arg-conf1/complex_top1.pdb
Bad element 'A' lines: 0

Example ligand resnames in complex_top1.pdb : ['AZO']


**pooling all complexes into a folder**

In [16]:
# optional: pooling all complexes into one folder (bash)
mkdir -p all_complexes
find dock_batch_flex -type f -name "complex_top1.pdb" -exec cp --parents {} all_complexes \;


SyntaxError: invalid syntax (3811481591.py, line 2)

**download everything**
*change the date*

In [ ]:
# DOWNLOAD EVERYTHING IN "Vinaflex" FOLDER
# THIS IS BASH
cd /home/s94m696/projects
du -sh vinaflex

# pack everything into one file
tar -czf vinaflex_02.08.2026.ver2.tar.gz vinaflex

ls -lh vinaflex_02.08.2026.ver2.tar.gz
